In [1]:
#Bibliotecas a importar
import numpy as np
import math as math

### Sesión 4: Valuación de Opciones en tiempo discreto

In [2]:
#t_op: Tipo de opcion
# 1: Opción europea tipo call
# 2: Opción europea tipo put
# 3: Opción digital tipo call
# 4: Opción digital tipo put
t_op = 1
T = 3 #Tiempo de vigencia de la opción (en años)
r = 0.06 #tasa anual libre de riesgo
k = 115 #Precio strike. Funciona como el salto en el caso de las digitales
S_0 = 120 #Precio del subyacente a t=0
n = 3 #Particiones del árbol

u = 1.7 #Factor de aumento en el precio
d = 0.8 #Factor de disminución de precio

# Probabilidad en la medida libre de riesgo

In [3]:
delta_t = T / n
p = (math.exp(r * delta_t) - d) / (u - d) #Probabilidad de aumento en el precio
p

0.2909294961615107

## Construcción del árbol

### Subyacente

Valor en el primer paso

In [4]:
#Matriz que almacena los valores del subyacente
arbol = np.zeros((n + 1, n + 1))
#El primer valor de la matriz es el subyacente a precio en t=0
arbol[0,0] = S_0
arbol #Visualziamos

array([[120.,   0.,   0.,   0.],
       [  0.,   0.,   0.,   0.],
       [  0.,   0.,   0.,   0.],
       [  0.,   0.,   0.,   0.]])

Valor del subyacente en el resto de los pasos

In [5]:
#For para llenar los valores de la matriz
for col in range(1, n +1):
    for ren in range(0, n +1):
        #Condicional para limitar matriz superior triangular
        if((col - ren) >= 0):
            arbol[ren, col] = S_0 *(( u ** (col - ren)) * (d ** (ren)))
arbol #Visualizamos

array([[120.  , 204.  , 346.8 , 589.56],
       [  0.  ,  96.  , 163.2 , 277.44],
       [  0.  ,   0.  ,  76.8 , 130.56],
       [  0.  ,   0.  ,   0.  ,  61.44]])

### Payoff

Primer paso

In [6]:
#Matriz que almacena el valor de los payoffs
payoffs = np.zeros((n+1, n+1))
#Sobreescribimos la última columna de la matriz de PAYOFSS
#IMPORTANTE: Agregar condicional para el caso del put (este es el caso particular del call)
if t_op==1:
    for ren in range(0, n+1):
        payoffs[ren, n] = max(arbol[ren, n] - k, 0)
elif t_op==2:
    for ren in range(0, n+1):
        payoffs[ren, n] = max(k-arbol[ren, n] , 0)
elif t_op==3:
    for ren in range(0, n+1):
        if (arbol[ren, n] > S_0):
            payoffs[ren, n] = k
        else:
            base[ren, n] = 0
elif t_op==4:
    for ren in range(0, n+1):
        if (arbol[ren, n] < S_0):
            payoffs[ren, n] = k
        else:
            payoffs[ren, n] = 0
payoffs #Visualizamos

array([[  0.  ,   0.  ,   0.  , 474.56],
       [  0.  ,   0.  ,   0.  , 162.44],
       [  0.  ,   0.  ,   0.  ,  15.56],
       [  0.  ,   0.  ,   0.  ,   0.  ]])

Valuamos backwards

In [7]:
#Sobreescribimos matriz de payoffs mediante el método iterativo
for i in range(1, n + 1):
    col = n - i
    for j in range(0, col +1): 
        payoffs[j, col] = math.exp(-r * delta_t) * (p * payoffs[j, col + 1] + (1 - p)*payoffs[j +1, col +1])
payoffs

array([[ 39.89301247, 102.00414978, 238.49707864, 474.56      ],
       [  0.        ,  17.88798521,  54.89707864, 162.44      ],
       [  0.        ,   0.        ,   4.26323898,  15.56      ],
       [  0.        ,   0.        ,   0.        ,   0.        ]])

## Valor de la opción

In [8]:
payoffs[0,0]

39.893012466730006

## Composición del portafolio

alfas

In [9]:
#Matriz que almacena los valores de las alfas
alfas = np.zeros((n, n ))
for ren in range(0,n):
    for col in range(0, n):
        #Condicional para limitar matriz superior triangular
        if((col - ren) >= 0):
            alfas[ren,col]=(payoffs[ren,col+1]-payoffs[ren+1,col+1])/(arbol[ren,col+1]-arbol[ren+1,col+1])
alfas #Visualizamos

array([[0.77885338, 1.        , 1.        ],
       [0.        , 0.58603981, 1.        ],
       [0.        , 0.        , 0.22511574]])

Betas

In [10]:
#Matriz que almacena los valores de las alfas
betas = np.zeros((n, n ))
for ren in range(0,n):
    for col in range(0, n):
        #Condicional para limitar matriz superior triangular
        if((col - ren) >= 0):
            betas[ren,col]=math.exp(-r * delta_t)*(payoffs[ren,col+1]-((payoffs[ren,col+1]-payoffs[ren+1,col+1])*arbol[ren,col+1])/(arbol[ren,col+1]-arbol[ren+1,col+1]))
betas #Visualizamos

array([[ -53.5693926 , -101.99585022, -108.30292136],
       [   0.        ,  -38.37183662, -108.30292136],
       [   0.        ,    0.        ,  -13.0256499 ]])